In [ ]:
#Imports and Styling

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set style for charts
sns.set_theme(style="whitegrid")
%matplotlib inline

%config InlineBackend.figure_format = 'retina'


In [ ]:
#Loading data and Error Handling

file_path = '../data/EURUSDX.csv'

if os.path.exists(file_path):
    df = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
    # Sort index to ensure time-series order
    df.sort_index(inplace=True)
    print(f"Successfully loaded {len(df)} rows.")
else:
    print(f"Error: {file_path} not found. Check your 'data' folder.")

df.head()


In [ ]:
#Data Cleaning and Audit

# Check for nulls
null_count = df.isnull().sum().sum()
if null_count > 0:
    print(f"Found {null_count} missing values. Applying Forward Fill...")
    df = df.ffill()
else:
    print("Dataset is clean (no missing values).")

# Stats for README
df.describe().transpose()


In [ ]:
#Log Returns

# Calculate both for comparison
df['Simple_Return'] = df['Close'].pct_change()
df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))

# Drop the first row (which becomes NaN after returns calculation)
df.dropna(inplace=True)

# Plotting the distribution (Histogram)
plt.figure(figsize=(10, 5))
sns.histplot(df['Log_Return'], bins=100, kde=True, color='blue')
plt.title('Distribution of EUR/USD Log Returns')
plt.show()


In [ ]:
#Technical Plot

# Calculate MAs
df['MA20'] = df['Close'].rolling(window=20).mean()
df['MA50'] = df['Close'].rolling(window=50).mean()

plt.figure(figsize=(14, 7))
plt.plot(df['Close'], label='EUR/USD Spot', alpha=0.5, color='gray')
plt.plot(df['MA20'], label='20-Day SMA (Fast)', color='orange', linewidth=1.5)
plt.plot(df['MA50'], label='50-Day SMA (Slow)', color='red', linewidth=2)

plt.title('EUR/USD Price Action & Trend Indicators', fontsize=15)
plt.ylabel('Exchange Rate')
plt.legend(frameon=True)
plt.show()


In [ ]:
print(df.columns)
df.head()


In [ ]:
# 1. Calculation: 20-day rolling volatility (annualized)
# Using a 252-day multiplier for standard business days in a year
window = 20
df['Volatility_20'] = df['Returns'].rolling(window=window).std() * np.sqrt(252)

# 2. Plotting
plt.figure(figsize=(12, 6))
plt.plot(df['Volatility_20'], label='20-Day Rolling Volatility', color='#2ecc71', lw=1.5)

# Add a horizontal line for the mean volatility to give context
mean_vol = df['Volatility_20'].mean()
plt.axhline(mean_vol, color='red', linestyle='--', alpha=0.7, label=f'Mean Vol ({mean_vol:.2%})')

# Formatting
plt.title('EUR/USD Realized Volatility Analysis (20-Day Annualized)', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Annualized Volatility', fontsize=12)
plt.legend(loc='upper left')

# Improve date formatting on the x-axis
plt.tight_layout()
plt.show()

# 3. Output summary for the report
print(f"Latest Realized Volatility: {df['Volatility_20'].iloc[-1]:.2%}")
print(f"Period Peak Volatility: {df['Volatility_20'].max():.2%}")


In [ ]:
# 1. Calculation: Cumulative returns (Growth of $1)
df['Cumulative_Returns'] = (1 + df['Returns'].fillna(0)).cumprod()

# 2. Plotting
plt.figure(figsize=(12, 6))
plt.plot(df['Cumulative_Returns'], label='Strategy Equity Curve', color='#3498db', lw=2)

# Add a "Break-even" reference line
plt.axhline(1.0, color='black', lw=1, linestyle='-', alpha=0.3)

# 3. Aesthetics & Annotations
plt.fill_between(df.index, 1.0, df['Cumulative_Returns'], 
                 where=(df['Cumulative_Returns'] >= 1.0), color='green', alpha=0.1)
plt.fill_between(df.index, 1.0, df['Cumulative_Returns'], 
                 where=(df['Cumulative_Returns'] < 1.0), color='red', alpha=0.1)

# Highlight final value
final_return = df['Cumulative_Returns'].iloc[-1]
plt.annotate(f'Final: {final_return:.4f}', 
             xy=(df.index[-1], final_return), 
             xytext=(df.index[-1], final_return + 0.01),
             arrowprops=dict(facecolor='black', shrink=0.05, width=1, headwidth=4),
             fontsize=10, fontweight='bold')

plt.title('EUR/USD Equity Curve: Cumulative Growth of $1', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Value (USD)', fontsize=12)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 4. Metric Summary
total_return = (final_return - 1) * 100
print(f"Total Period Return: {total_return:.2f}%")


In [ ]:
# 1. Calculation: Drawdown from Peak
rolling_max = df['Cumulative_Returns'].cummax()
df['Drawdown'] = (df['Cumulative_Returns'] / rolling_max) - 1

# 2. Plotting the "Underwater" Chart
plt.figure(figsize=(12, 6))

# Fill the area to create the "Underwater" effect
plt.fill_between(df.index, df['Drawdown'], 0, color='#e74c3c', alpha=0.3, label='Drawdown Area')
plt.plot(df['Drawdown'], color='#c0392b', lw=1.5, label='Drawdown Line')

# 3. Identify and Annotate the Maximum Drawdown (MDD)
max_dd = df['Drawdown'].min()
max_dd_date = df['Drawdown'].idxmin()

plt.scatter(max_dd_date, max_dd, color='black', s=50, zorder=5)
plt.annotate(f'Max Drawdown: {max_dd:.2%}', 
             xy=(max_dd_date, max_dd), 
             xytext=(max_dd_date, max_dd - 0.02),
             arrowprops=dict(facecolor='black', shrink=0.05, width=1, headwidth=4),
             ha='center', fontweight='bold')

# 4. Aesthetics
plt.title('EUR/USD Maximum Drawdown Analysis (Underwater Plot)', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Drawdown (%)', fontsize=12)
plt.ylim(df['Drawdown'].min() - 0.05, 0.05) # Give some padding at the bottom
plt.axhline(0, color='black', lw=1, alpha=0.5)
plt.grid(True, alpha=0.2, linestyle='--')
plt.legend(loc='lower left')

plt.tight_layout()
plt.show()

# 5. Print Analytics
print(f"Maximum Drawdown: {max_dd:.2%}")
print(f"Peak Value before MDD: {rolling_max.loc[max_dd_date]:.4f}")


In [ ]:
# 1. Calculation: 20-Day Rolling Sharpe Ratio
# We assume a 0% risk-free rate for FX, but we'll label it for professional clarity
rf = 0 
window = 20
rolling_mean = df['Returns'].rolling(window=window).mean()
rolling_std = df['Returns'].rolling(window=window).std()

# Annualized Sharpe = (Mean / Std) * sqrt(Annualization Factor)
df['Sharpe_20'] = ((rolling_mean - rf) / rolling_std) * np.sqrt(252)

# 2. Plotting
plt.figure(figsize=(12, 6))
plt.plot(df['Sharpe_20'], color='#8e44ad', lw=1.5, label='20-Day Rolling Sharpe')

# 3. Add Contextual Benchmarks
plt.axhline(0, color='black', lw=1, linestyle='-') # Break-even
plt.axhline(1, color='orange', lw=1, linestyle='--', alpha=0.6, label='Good (1.0)')
plt.axhline(2, color='green', lw=1, linestyle='--', alpha=0.6, label='Excellent (2.0)')

# Fill the area below 0 to highlight "Risk without Reward"
plt.fill_between(df.index, df['Sharpe_20'], 0, where=(df['Sharpe_20'] < 0), 
                 color='red', alpha=0.1)

# 4. Aesthetics
plt.title('EUR/USD Risk-Adjusted Performance (Rolling Sharpe Ratio)', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Annualized Sharpe Ratio', fontsize=12)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()

# 5. Summary Analytics
print(f"Current Sharpe Ratio: {df['Sharpe_20'].iloc[-1]:.2f}")
print(f"Average Sharpe (Period): {df['Sharpe_20'].mean():.2f}")


In [ ]:
# 1. Configuration & Standardization
asset_list = {
    'EURUSD': '../data/EURUSDX.csv',
    'USDJPY': '../data/USDJPYX.csv',
    'AAPL': '../data/AAPL.csv',
    'MSFT': '../data/MSFT.csv'
}

returns_df = pd.DataFrame()

# 2. Efficiently building the Returns Matrix
for name, path in asset_list.items():
    try:
        # Load data, ensuring column names are handled (case-insensitive)
        temp_df = pd.read_csv(path, parse_dates=True, index_col=0)
        temp_df.columns = [col.capitalize() for col in temp_df.columns]
        
        # Calculate daily returns and assign to master dataframe
        returns_df[name] = temp_df['Close'].pct_change()
    except Exception as e:
        print(f"Error loading {name}: {e}")

# 3. Handling Time-Series Alignment
# Since FX trades 24/5 and Stocks 9:30-4:00, we use an inner join (dropna) 
# to ensure we only compare timestamps where all assets were active.
returns_df.dropna(inplace=True)

# 4. Display Statistical Summary
print("--- Cross-Asset Returns Matrix Summary ---")
print(f"Total Overlapping Trading Days: {len(returns_df)}")
display(returns_df.describe().transpose()[['mean', 'std', 'min', 'max']])


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Compute the Correlation Matrix
corr_matrix = returns_df.corr()

# 2. Create a Mask for the Upper Triangle
# A correlation matrix is symmetrical; showing both sides is redundant visual noise.
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# 3. Plotting
plt.figure(figsize=(10, 8))

# We use 'coolwarm' because it centers at 0 (white). 
# Red = Highly Correlated (Risk overlap)
# Blue = Inversely Correlated (Hedge)
sns.heatmap(corr_matrix, 
            mask=mask, 
            annot=True, 
            fmt=".2f", 
            cmap='coolwarm', 
            vmin=-1, vmax=1, center=0,
            square=True, 
            linewidths=.5, 
            cbar_kws={"shrink": .8})

# 4. Aesthetics
plt.title('Cross-Asset Return Correlation Heatmap', fontsize=15, fontweight='bold', pad=20)
plt.xticks(rotation=45)
plt.yticks(rotation=0)

plt.tight_layout()
plt.show()

# 5. Quant Insight Output
# Let's programmatically find the highest and lowest correlation pairs
sol = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)).stack().sort_values()
print(f"Lowest Correlation Pair: {sol.index[0]} ({sol.iloc[0]:.2f})")
print(f"Highest Correlation Pair: {sol.index[-1]} ({sol.iloc[-1]:.2f})")


In [ ]:
# 1. Calculation: Trend Regimes
# Logic: 1 = Bullish (Golden Cross), -1 = Bearish (Death Cross)
df['Trend_Regime'] = np.where(df['MA20'] > df['MA50'], 1, -1)

# 2. Visualization: Overlaying Regimes on Price
plt.figure(figsize=(14, 7))

# Plot the underlying price and MAs
plt.plot(df['Close'], color='black', alpha=0.3, label='EUR/USD Price')
plt.plot(df['MA20'], color='blue', lw=1, label='20-Day MA', alpha=0.8)
plt.plot(df['MA50'], color='orange', lw=1, label='50-Day MA', alpha=0.8)

# 3. Create Regime Background Shading
# Green for Uptrend, Red for Downtrend
plt.fill_between(df.index, df['Close'].min(), df['Close'].max(), 
                 where=(df['Trend_Regime'] == 1), 
                 color='green', alpha=0.1, label='Bullish Regime')
plt.fill_between(df.index, df['Close'].min(), df['Close'].max(), 
                 where=(df['Trend_Regime'] == -1), 
                 color='red', alpha=0.1, label='Bearish Regime')

# Aesthetics
plt.title('Market Regime Analysis: Trend Classification', fontsize=14, fontweight='bold')
plt.ylabel('Price')
plt.legend(loc='upper left', frameon=True)
plt.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()

# 4. Verification Output
print("--- Recent Regime States ---")
print(df[['Close', 'MA20', 'MA50', 'Trend_Regime']].tail())


In [ ]:
# 1. Setup Plot
plt.figure(figsize=(14, 7))

# 2. Add Reference Moving Averages (Subtle)
# These show the user WHY the background color is changing
plt.plot(df['MA20'], color='blue', lw=1, alpha=0.4, label='MA20 (Fast)')
plt.plot(df['MA50'], color='orange', lw=1, alpha=0.4, label='MA50 (Slow)')

# 3. Plot Primary Price Data
plt.plot(df['Close'], color='#2c3e50', lw=2, label='Close Price')

# 4. Apply Shaded Regimes
# We use axvspan logic via fill_between with a better y-axis anchor
y_min = df['Close'].min() * 0.99
y_max = df['Close'].max() * 1.01

plt.fill_between(df.index, y_min, y_max,
                 where=(df['Trend_Regime'] == 1), 
                 color='green', alpha=0.15, label='Bullish Regime')

plt.fill_between(df.index, y_min, y_max,
                 where=(df['Trend_Regime'] == -1), 
                 color='red', alpha=0.15, label='Bearish Regime')

# 5. Formatting & Presentation
plt.title('EUR/USD Market State Analysis: Trend Regime Identification', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price', fontsize=12)
plt.ylim(y_min, y_max) # Lock the y-axis for a tighter look
plt.legend(loc='upper left', frameon=True, shadow=True)
plt.grid(True, alpha=0.2, linestyle='--')

plt.tight_layout()
plt.show()


In [ ]:
# 1. Calculation: Define a dynamic Volatility Threshold
# We use a 50-day rolling median to avoid look-ahead bias
df['Vol_Threshold'] = df['Volatility_20'].rolling(window=50).median()

# 2. Classification: High vs Low Volatility
# Using np.where is more efficient for binary classification
df['Vol_Regime'] = np.where(df['Volatility_20'] > df['Vol_Threshold'], 'High Vol', 'Low Vol')

# 3. Visualization: Volatility vs. Threshold
plt.figure(figsize=(14, 6))
plt.plot(df['Volatility_20'], label='20-Day Annualized Vol', color='#8e44ad', lw=2)
plt.plot(df['Vol_Threshold'], label='50-Day Rolling Threshold', color='black', linestyle='--', alpha=0.6)

# Highlight High Volatility periods
plt.fill_between(df.index, df['Volatility_20'], df['Vol_Threshold'], 
                 where=(df['Volatility_20'] > df['Vol_Threshold']), 
                 color='#e74c3c', alpha=0.2, label='High Vol Regime')

# Aesthetics
plt.title('Volatility Regime Detection (Dynamic Threshold)', fontsize=14, fontweight='bold')
plt.ylabel('Annualized Volatility')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 4. Verification Output
print("--- Recent Volatility State ---")
print(df[['Volatility_20', 'Vol_Threshold', 'Vol_Regime']].tail())


In [ ]:
# 1. Plot Setup
plt.figure(figsize=(14, 6))

# 2. Plot Realized Volatility and the Dynamic Threshold
plt.plot(df['Volatility_20'], label='20-Day Realized Volatility', color='#8e44ad', lw=2)
plt.plot(df['Vol_Threshold'], label='50-Day Rolling Median (Threshold)', 
         color='black', linestyle='--', alpha=0.6)

# 3. Highlight "Stressed" Market States (High Vol)
# We shade the area between the volatility and its threshold when it's exceeded
plt.fill_between(df.index, df['Volatility_20'], df['Vol_Threshold'], 
                 where=(df['Volatility_20'] > df['Vol_Threshold']), 
                 color='#e74c3c', alpha=0.3, label='High Vol Regime')

# 4. Aesthetics and Presentation
plt.title('EUR/USD Volatility Regime Detection', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Annualized Volatility (%)')
plt.legend(loc='upper left', frameon=True)

# Format y-axis as percentage for clarity
from matplotlib.ticker import PercentFormatter
# Note: Use PercentFormatter(1.0) if your vol is 0.15 for 15%
# Or leave as is if already scaled to whole numbers.

plt.grid(True, alpha=0.2, linestyle='--')
plt.tight_layout()
plt.show()


In [ ]:
# 1. Create Descriptive Labels for the Matrix
# Map the numbers back to professional terminology
trend_map = {1: 'Bullish', -1: 'Bearish', 0: 'Neutral'}
df['Trend_Label'] = df['Trend_Regime'].map(trend_map)

# 2. Combine into a "Market State"
df['Market_State'] = df['Trend_Label'] + " / " + df['Vol_Regime']

# 3. Aggregate Performance by Market State
# We annualize the mean (x252) and std (xsqrt(252)) for professional comparison
state_stats = df.groupby('Market_State')['Returns'].agg(['mean', 'std', 'count'])

# Rename columns for clarity
state_stats.columns = ['Avg Daily Return', 'Daily Volatility', 'Days in State']

# 4. Add Advanced Quant Metrics to the Matrix
state_stats['Annualized Return (%)'] = state_stats['Avg Daily Return'] * 252 * 100
state_stats['Annualized Vol (%)'] = state_stats['Daily Volatility'] * np.sqrt(252) * 100
state_stats['Sharpe Ratio'] = (state_stats['Annualized Return (%)'] / state_stats['Annualized Vol (%)'])

# 5. Clean up the Presentation
# We sort by Sharpe to see which regime is most "efficient"
final_report = state_stats[['Days in State', 'Annualized Return (%)', 'Annualized Vol (%)', 'Sharpe Ratio']]
final_report = final_report.sort_values(by='Sharpe Ratio', ascending=False)

# Display with a background gradient for instant visual analysis
display(final_report.style.background_gradient(cmap='RdYlGn', subset=['Sharpe Ratio'])
                          .format("{:.2f}", subset=['Annualized Return (%)', 'Annualized Vol (%)', 'Sharpe Ratio']))


In [ ]:
## Regime Analysis Insights

- Uptrend + Low Vol regimes show more stable positive returns
- High Vol regimes correspond to larger drawdowns
- Trend regimes align with moving average crossovers
- Volatility regimes signal risk conditions, not direction

This demonstrates how market behavior changes over time and why 
strategies must adapt to regimes instead of assuming constant behavior.